# Calculating Network Error with Loss

## Categorical Cross-Entropy Loss

![Categorical Cross-Entropy Loss](./categorical-ce-loss.png)

Example:

In [1]:
import math

softmax_output = [0.7, 0.1, 0.2]
target_output = [1,0,0]

loss = -(math.log(softmax_output[0]) * target_output[0] +
         math.log(softmax_output[1]) * target_output[1] +
         math.log(softmax_output[2]) * target_output[2])

print(loss)

0.35667494393873245


But technically, `target_output[1]` and `target_output[2]` are both zero (since they are not favored classes). And `target_output[0]` is 1, so it can be omitted as well. Thus, we can go for a more simplified solution.

In [ ]:
import numpy as np


softmax_outputs = np.array([
    [0.7, 0.1, 0.2],
    [0.1, 0.5, 0.4],
    [0.02, 0.9, 0.8],
])
class_targets = [0, 1, 1]

print(softmax_outputs[
    range(len(softmax_outputs)), class_targets
])

[0.7 0.5 0.9]


In [ ]:
import numpy as np

softmax_outputs = np.array([
    [0.7, 0.1, 0.2],
    [0.1, 0.5, 0.4],
    [0.02, 0.9, 0.8],
])
class_targets = [0, 1, 1]

print(-np.log(softmax_outputs[
    range(len(softmax_outputs)), class_targets
]))

[0.35667494 0.69314718 0.10536052]


In [4]:
import numpy as np

softmax_outputs = np.array([
    [0.7, 0.1, 0.2],
    [0.1, 0.5, 0.4],
    [0.02, 0.9, 0.8],
])
class_targets = [0, 1, 1]

neg_log = -np.log(softmax_outputs[
    range(len(softmax_outputs)), class_targets
])
average_loss = np.mean(neg_log)
print(average_loss)

0.38506088005216804


## Clipping because of limits log(0)

The log(0) is undefined, and it approaches negative infinity. However, we can clip the largest possible loss by adding a small value epsilon for numerical stability.

`y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)`

## Final Cross-Entropy Loss Class

In [10]:
import numpy as np


class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss


class CategoricalCrossEntropyLoss(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)

        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
                range(samples),
                y_true
            ]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(
                y_pred_clipped * y_true,
                axis=1
            )
        
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods


In [13]:
import numpy as np

softmax_outputs = np.array([
    [0.7, 0.1, 0.2],
    [0.1, 0.5, 0.4],
    [0.02, 0.9, 0.8],
])
class_targets = np.array([0, 1, 1])


loss_function = CategoricalCrossEntropyLoss()
loss = loss_function.calculate(softmax_outputs, class_targets)
print(loss)

0.38506088005216804


# Combining everything

In [4]:
from nnfs.datasets import spiral_data
import numpy as np
import nnfs

class LayerDense:
    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        # initialize weights and biases 
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases


class ActivationReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)


class ActivationSoftmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities
    
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss
    
class CategoricalEntropyLoss(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)

        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
                range(samples),
                y_true
            ]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(
                y_pred_clipped * y_true,
                axis=1
            )

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

In [5]:
# create dataset
x, y = spiral_data(samples=100, classes=3)

dense1 = LayerDense(2, 3)

activation1 = ActivationReLU()

dense2 = LayerDense(3, 3)

activation2 = ActivationSoftmax()

loss_function = CategoricalEntropyLoss()

dense1.forward(x)

activation1.forward(dense1.output)

dense2.forward(activation1.output)

activation2.forward(dense2.output)

print(activation2.output[:5])

loss = loss_function.calculate(activation2.output, y)
print('Loss:', loss)

[[0.33333333 0.33333333 0.33333333]
 [0.33333377 0.33333304 0.33333318]
 [0.33333435 0.33333262 0.33333302]
 [0.33333507 0.33333206 0.33333287]
 [0.33333586 0.33333133 0.33333281]]
Loss: 1.098615968055465
